### Set up nerf_template

In [ ]:
!pip install numpy==1.24.4 --force-reinstall --quiet

import os
os.kill(os.getpid(), 9)

In [ ]:
!pip install -q torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 \
    -f https://download.pytorch.org/whl/torch_stable.html

!pip install -q triton scikit-image

In [ ]:
%cd /content
!git clone https://github.com/ashawkey/nerf_template.git
%cd nerf_template
!pip install -q -r requirements.txt

* Patch the nerf_template imports/metric computations

In [ ]:
import pathlib, re

f = pathlib.Path('nerf/utils.py')

txt = f.read_text()
pattern = r"try:.*?structural_similarity_index_measure.*?ssim.*?\n"
replacement = "from skimage.metrics import structural_similarity as structural_similarity_index_measure\n"
patched = re.sub(pattern, replacement, txt, flags=re.S)
f.write_text(patched)

txt = f.read_text()
txt = re.sub(
    r"# try out torch 2\.0[\s\S]*?model = torch\.compile\(model\)",
    (
        "# torch.compile disabled (Python 3.11+ unsupported)\n"
        "# try out torch 2.0\n"
        "# if torch.__version__[0] == '2':\n"
        "#     model = torch.compile(model)"
    ),
    txt
)
txt = re.sub(
    r"(class Trainer\(object\):\s+def __init__\([^)]*\):)",
    r"\1\n        # ensure log_ptr exists for __del__\n        self.log_ptr = None",
    txt,
    flags=re.MULTILINE
)
f.write_text(txt)

txt = f.read_text()
ssim_patch = r"""
    def update(self, preds, truths):
        if torch.is_tensor(preds):
            preds = preds.detach().cpu().numpy()
        if torch.is_tensor(truths):
            truths = truths.detach().cpu().numpy()

        if preds.ndim == 4:
            preds = preds[0]
        if truths.ndim == 4:
            truths = truths[0]

        preds = np.clip(preds, 0.0, 1.0)
        truths = np.clip(truths, 0.0, 1.0)

        ssim = structural_similarity_index_measure(preds, truths, channel_axis=-1, data_range=1.0)

        self.V += ssim
        self.N += 1
        return ssim
"""
txt = re.sub(
    r"def update\(self, preds, truths\):\s+preds, truths = self\.prepare_inputs\([^)]+\)[\s\S]+?self\.N \+= 1",
    ssim_patch.strip(),
    txt
)
f.write_text(txt)

In [ ]:
import pathlib, re

f = pathlib.Path('/content/nerf_template/nerf/provider.py')
txt   = f.read_text()
txt = re.sub(
    r"(if self\.training:\s*)(images = self\.images\[index,\s*rays\['j'\],\s*rays\['i'\]\]\.float\(\)\.to\(self\.device\) / 255)",
    r"""\1
                rays_i = rays['i'].cpu()
                rays_j = rays['j'].cpu()
                images = self.images[index, rays_j, rays_i].float().to(self.device) / 255""",
    txt,
    flags=re.MULTILINE
)
f.write_text(txt)

### Set up the dataset

In [ ]:
import os
import zipfile
import json
import shutil
import threading
from glob import glob
from tqdm import tqdm
import numpy as np
import random
from pathlib import Path

In [ ]:
SCENE = "0097"

ZIP_PATH   = f"/content/drive/MyDrive/thesis/Data_Preparation/Original_Datasets/{SCENE}.zip"
SCENE_DIR  = "/content/{SCENE}"

with zipfile.ZipFile(ZIP_PATH, 'r') as z:
    z.extractall(SCENE_DIR)

* Convert the colmap data into a format that nerf_template can use

In [ ]:
!python scripts/colmap2nerf.py \
    --colmap_text {SCENE_DIR}/poses/colmap_text \
    --images      {SCENE_DIR}/images \
    --hold 10

In [ ]:
TRANSFORM_JSON = f"{SCENE_DIR}/transforms_train.json"

train_frames = json.load(open(TRANSFORM_JSON, "r"))["frames"]
len(train_frames)

### NeRF training

In [ ]:
WORKSPACE = f"/content/workspace"

!rm -rf {WORKSPACE}

In [ ]:
dummy = 1

!python main.py {SCENE_DIR} \
    --data_format nerf \
    --workspace {WORKSPACE} \
    -O \
    --bound 1.0 \
    --scale 0.33 \
    --iters 27000 \
    --save_cnt 10 \
    --eval_cnt 30   \
    --test_no_video \
    --test_no_mesh

### Checkpoint saving, NeRF rendering and dataset saving

In [ ]:
CHECKPOINTS_DRIVE_PATH = f"/content/drive/MyDrive/thesis/Data_Preparation/Checkpoints_Metrics/{SCENE}"

os.makedirs(CHECKPOINTS_DRIVE_PATH, exist_ok=True)

CHECKPOINT = f"{WORKSPACE}/checkpoints/ngp_ep0300.pth"
LOGS = f"{WORKSPACE}/log_ngp.txt"

shutil.copy(CHECKPOINT, CHECKPOINTS_DRIVE_PATH)
shutil.copy(LOGS, CHECKPOINTS_DRIVE_PATH)

#### Include all frames into test frames to render them

By default, the nerf_template will render the test frames when performing the evaluation.

In [ ]:
dummy = 1

!rm {SCENE_DIR}/transforms.json
!rm {SCENE_DIR}/transforms_train.json
!rm {SCENE_DIR}/transforms_test.json
!rm {SCENE_DIR}/transforms_val.json

In [ ]:
dummy = 1

!python scripts/colmap2nerf.py \
    --colmap_text {SCENE_DIR}/poses/colmap_text \
    --images      {SCENE_DIR}/images \
    --hold 1

In [ ]:
TRANSFORM_JSON_TEST = f"{SCENE_DIR}/transforms_test.json"

test_frames = json.load(open(TRANSFORM_JSON_TEST, "r"))["frames"]
len(test_frames)

In [ ]:
train_json = json.load(open(TRANSFORM_JSON, "r"))
train_json["frames"] = test_frames

with open(TRANSFORM_JSON, "w") as f:
    json.dump(train_json, f, indent=2)

train_frames = json.load(open(TRANSFORM_JSON, "r"))["frames"]
len(train_frames)

In [ ]:
VAL_DIR = f"{WORKSPACE}/validation"

!rm -rf {VAL_DIR}

* Same settings as the training, the framework will load the last checkpoint and only perform the test

In [ ]:
dummy = 1

!python main.py {SCENE_DIR} \
    --data_format nerf \
    --workspace {WORKSPACE} \
    -O \
    --bound 1.0 \
    --scale 0.33 \
    --iters 27000 \
    --save_cnt 6 \
    --eval_cnt 3   \
    --test_no_video \
    --test_no_mesh

* Format the data

In [ ]:
!rm -rf /content/output

OUTPUT_DIR = "/content/output"
IMAGES_DIR = f"{OUTPUT_DIR}/images"
DEPTHS_DIR = f"{OUTPUT_DIR}/depths"

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(IMAGES_DIR, exist_ok=True)
os.makedirs(DEPTHS_DIR, exist_ok=True)

with open(TRANSFORM_JSON, 'r') as f:
    data = json.load(f)

frames = data["frames"]
camera_metadata = {k: v for k, v in data.items() if k != "frames"}

In [ ]:
PREFIX = "ngp_ep0300"

camera_list = []
for i, frame in enumerate(frames):
    original_name = Path(frame["file_path"]).name
    index = i + 1
    padded_index = f"{index:04d}"

    rgb_path = Path(VAL_DIR) / f"{PREFIX}_{padded_index}_rgb.png"
    depth_path = Path(VAL_DIR) / f"{PREFIX}_{padded_index}_depth.png"

    out_rgb = f"{IMAGES_DIR}/" + original_name.replace(".jpg", ".color.png")
    out_depth = f"{DEPTHS_DIR}/" + original_name.replace(".jpg", ".depth.png")

    if rgb_path.exists():
        shutil.copy(rgb_path, out_rgb)
    else:
        print(f"[WARN] Missing RGB: {rgb_path}")

    if depth_path.exists():
        shutil.copy(depth_path, out_depth)
    else:
        print(f"[WARN] Missing depth: {depth_path}")

    camera_list.append({
        "file_path": original_name,
        "transform_matrix": frame["transform_matrix"]
    })

camera_output = {
    **camera_metadata,
    "frames": camera_list
}
with open(Path(OUTPUT_DIR) / "cameras.json", "w") as f:
    json.dump(camera_output, f, indent=2)

print(f"Exported {len(camera_list)} frames to: {OUTPUT_DIR}")

In [ ]:
ZIP_NAME = f"{SCENE}.zip"
DRIVE_DIR = "/content/drive/MyDrive/thesis/datasets/exports"

shutil.make_archive(base_name="/content/" + ZIP_NAME.replace(".zip", ""),
                    format="zip",
                    root_dir=OUTPUT_DIR,
                    base_dir=".")

shutil.copy(f"/content/{ZIP_NAME}", os.path.join(DRIVE_DIR, ZIP_NAME))

print(f"Zipped contents of {OUTPUT_DIR} and copied to {DRIVE_DIR}/{ZIP_NAME}")